# Sammensetninger

In [56]:
import sqlite3
import pandas as pd
import dhlab as dh
import sys
import math
import ipywidgets as widgets
from IPython.display import display, Markdown

In [3]:
db = "/mnt/disk1/Github/hyphens.db"
uni = "/mnt/disk2/NB-ngram-assoc/unigram-one-row.db"
hyp = "/mnt/disk1/Github/total.db"

In [4]:
def query(db, sql, param = ()):
    res = []
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        try:
            cur.execute(sql, param)
            res = cur.fetchall()
        except:
            print('en feil oppstod:', str(sys.exc_info()[1]) )
    return res

def pdquery(db, sql, params = ()):
    res = pd.DataFrame()
    with sqlite3.connect(db) as con:
        try:
            res = pd.read_sql_query(sql, con, params = params)
        except:
            print('en feil oppstod:', str(sys.exc_info()[1]) )
    return res

In [5]:
def analyse(w, top=True):
    res = dict()
    for (f,s) in splits(w):
        fx = query(uni,f"select first, freq from unigram where lang = 'nob' and first = '{f}'" )
        sx = query(uni,f"select first, freq from unigram where lang = 'nob' and first = '{s}'" )
        try:
            fa = fx[0][1]
            sa = sx[0][1]
            #M = (fa + sa)/2
            #sd = math.sqrt( ((fa - M)**2 + (sa - M)**2)/2)
            #percent = abs(fa - sa)/sd
            res[(f,s, f"{f}{s}")] = (fa, sa, math.log(fa*sa), sa/fa)
        except:
            pass
    if top:
        result = pd.DataFrame((pd.DataFrame(res, index=['first_freq','second_freq','log_prod', 'sd'])
            .transpose()
            .sort_values(by='log_prod', ascending=False)).iloc[0]).transpose()
    else:
        result = (pd.DataFrame(res, index=['first_freq','second_freq','log_prod', 'sd'])
            .transpose()
            .sort_values(by='log_prod', ascending=False))
    return result

In [6]:
def compose(x, top=True):
    try:
        if top:
            res = pd.DataFrame(pdquery(hyp, f"select * from hyphendata where word='{x}'").sort_values(by="freq", ascending=False).iloc[0]).transpose()
        else:
            res = pd.DataFrame(pdquery(hyp, f"select * from hyphendata where word='{x}'").sort_values(by="freq", ascending=False))
    except:
        res = pd.DataFrame([(0,"","",x)], columns = "freq first second word".split())
    return res
                        

In [7]:
def splits(x):
    mains = [(x[0:i],x[i:]) for i in range(1,len(x))]
    #print(mains)
    fuge = []
    for prefix in mains:
        #print(prefix)
        if len(prefix[0]) > 1 and prefix[0].endswith('e') or prefix[0].endswith('s'):
            fuge.append((prefix[0][:-1], prefix[1]))
    return mains + fuge

In [50]:

def show_words(change = '', top = True):
    words = change.split()
    #print(words)
    res = pd.DataFrame()
    res2 = pd.DataFrame()
    try:
        res = pd.concat([analyse(x, top=top) for x in words])
        res2 = pd.concat([compose(x, top=top) for x in words])
    except:
        pass
    display(Markdown("## ngram"))
    display(res)
    display(Markdown("## hyphen-data"))
    display(res2)
    

In [51]:
display(Markdown("Legg til en eller flere ord, data oppdateres dynamisk"))
word = widgets.Text("nasjonalbiblioteket")
top = widgets.Checkbox(value=True, description="Klikk for å vise alt" )

In [52]:
out = widgets.interactive_output(show_words, {'change': word, 'top':top})

In [58]:
widgets.VBox([word, top])

NameError: name 'top' is not defined

In [59]:
widgets.HBox([out])

In [57]:
#widgets.HBox([widgets.VBox([word]), out])